# 📊 Evaluate Vietnamese-English Model

Evaluate and test the VI → EN translation model.

---

## 1. ⚙️ Setup

In [ ]:
!git clone https://github.com/TranKien2005/EV_Translate_Modle_NLP_Project.git
%cd EV_Translate_Modle_NLP_Project

In [ ]:
!pip install -q sentencepiece sacrebleu google-generativeai python-dotenv tqdm pyyaml

In [ ]:
GEMINI_API_KEY = "YOUR_GEMINI_API_KEY_HERE"

with open('.env', 'w') as f:
    f.write(f'GEMINI_API_KEY={GEMINI_API_KEY}\n')
print('✓ .env created')

## 2. 🔧 Configure Paths

In [ ]:
import yaml

# Path to your model dataset
KAGGLE_INPUT_MODEL = '/kaggle/input/vi-en-model'  # CHANGE THIS

CONFIG_FILE = 'config/config_vi_en.yaml'

with open(CONFIG_FILE, 'r') as f:
    cfg = yaml.safe_load(f)

cfg['paths'] = {
    'data_dir': '/kaggle/working/data',
    'checkpoint_dir': '/kaggle/working/checkpoints_vi_en',
    'log_dir': '/kaggle/working/logs_vi_en'
}

with open(CONFIG_FILE, 'w') as f:
    yaml.dump(cfg, f, default_flow_style=False, allow_unicode=True)

print('✓ Config paths updated')

In [ ]:
import shutil

!mkdir -p /kaggle/working/checkpoints_vi_en/tokenizers

# Copy model files
shutil.copy(f'{KAGGLE_INPUT_MODEL}/best_model.pt', '/kaggle/working/checkpoints_vi_en/best_model.pt')
shutil.copy(f'{KAGGLE_INPUT_MODEL}/tokenizer_vi.model', '/kaggle/working/checkpoints_vi_en/tokenizers/tokenizer_vi.model')
shutil.copy(f'{KAGGLE_INPUT_MODEL}/tokenizer_en.model', '/kaggle/working/checkpoints_vi_en/tokenizers/tokenizer_en.model')

print('✓ Model files copied')

## 3. 📥 Load Model

In [ ]:
import sys
sys.path.insert(0, '.')

from src.evaluate import load_translator
from src.config import load_config

config = load_config('config/config_vi_en.yaml')

translator = load_translator(
    checkpoint_path=str(config.paths.checkpoint_dir / 'best_model.pt'),
    vocab_src_path=str(config.paths.checkpoint_dir / 'tokenizers/tokenizer_vi.model'),
    vocab_tgt_path=str(config.paths.checkpoint_dir / 'tokenizers/tokenizer_en.model'),
    config_path='config/config_vi_en.yaml'
)

print('✓ Translator loaded')

## 4. 🧪 Test Translations

In [ ]:
# Test sentences
test_sentences = [
    "Xin chào, bạn khỏe không?",
    "Tôi yêu Việt Nam.",
    "Hôm nay thời tiết rất đẹp.",
    "Cảm ơn bạn rất nhiều.",
    "Tôi đang học tiếng Anh."
]

print("🇻🇳 → 🇬🇧 Vietnamese to English Translations")
print("="*50)

for vn in test_sentences:
    en = translator.translate(vn)
    print(f"\nVI: {vn}")
    print(f"EN: {en}")

## 5. 📝 Interactive Mode

In [ ]:
# Interactive translation (type 'quit' to exit)
print("Interactive VI → EN Translation")
print("Type Vietnamese text and press Enter")
print("Type 'quit' to exit")
print("="*50)

while True:
    text = input("\nVI: ").strip()
    if text.lower() == 'quit':
        break
    translation = translator.translate(text)
    print(f"EN: {translation}")